In [2]:
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from dotenv import load_dotenv
import nest_asyncio
import asyncio

# Load environment variables (GOOGLE_API_KEY)
load_dotenv()

# Define variables required for Session setup and Agent execution
APP_NAME = "Google_Search_agent"
USER_ID = "user1234"
SESSION_ID = "1234"


# Define Agent with access to search tool
root_agent = LlmAgent(
   name="basic_search_agent",
   model="gemini-2.0-flash-exp",
   description="Agent to answer questions using Google Search.",
   instruction="I can answer your questions by searching the internet. Just ask me anything!",
   tools=[google_search]  # Google Search is a pre-built tool to perform Google searches.
)

# Agent Interaction
async def call_agent(query):
   """
   Helper function to call the agent with a query.
   """

   # Session and Runner
   session_service = InMemorySessionService()
   session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
   runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

   content = types.Content(role='user', parts=[types.Part(text=query)])
   
   print(f"\n--- Running Query: {query} ---")
   
   # Use async iteration for run_async
   async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
       if event.is_final_response() and event.content:
           if hasattr(event.content, 'text') and event.content.text:
               final_response = event.content.text
           elif event.content.parts:
               text_parts = [part.text for part in event.content.parts if part.text]
               final_response = "".join(text_parts)
           
           print("\n" + "=" * 80)
           print("Agent Response:")
           print("=" * 80)
           print(final_response)
           print("=" * 80)

# Apply nest_asyncio to allow running async code in Jupyter
nest_asyncio.apply()

# Run the agent
asyncio.run(call_agent("what's the latest AI news?"))


--- Running Query: what's the latest AI news? ---

Agent Response:
Here's a summary of the latest AI news:

*   **AI Integration Across Industries:** Disney is embedding generative AI across its entire operating structure to enhance efficiency in content creation and personalized guest experiences. Several reports discuss AI being used in retail, clinical trials and other industries.
*   **AI for Medicine:** Researchers are developing AI-driven frameworks to improve the accuracy of evidence-based medicine and tools to uncover hidden genetic networks driving cancer.
*   **AI Hardware Advancements:** There's news about tiny chips that could change the future of quantum computing. Nvidia is partnering with AI hardware startup Groq.
*   **Human-AI Collaboration:** A study found that hybrid human + AI teams outperform fully autonomous agents.
*   **AI Security:** A report indicates that AI adoption is outpacing security controls, leaving firms exposed to AI-enabled social engineering attac